# LAB 5 - Matheus Gomes Maia
## ATIVIDADE
5 palavras = ["lista", "com", "varias", "gato", "mano"]


* 1 Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que? (20 pts.)

* 2 De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a qualidade dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta. (25 pts.)


## SETUP

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import math
import re
from scipy.optimize import minimize_scalar
import seaborn as sns
from nltk.corpus import stopwords
import operator

pd.set_option("display.max_rows",40)

%matplotlib inline
nltk.download('punkt')

nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/maia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/maia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [86]:
data = pd.read_csv('../results_fornecida.csv')
data = data.drop_duplicates(subset='url', keep='last')


#Pre process news
# Tokenize, Join and Filter
# Words into a new data Frame
txt = [str(news) for news in data["text"].tolist()]
words = [nltk.word_tokenize(sentence) for sentence in txt]
words = [item for sublist in words for item in sublist] 
words = [word.lower() for word in words if (word.isalpha() and len(word)) > 2] 

# New Data frame with Word, Frequency and Ranking columns 
words_df = pd.DataFrame(words, columns=['word']) #All words
word_counts = words_df.word.value_counts().reset_index() #Join by word. Adds Frequency
word_counts.columns = ['Word', 'Freq'] #Naming columns 
word_counts['word_rank'] = word_counts.Freq.rank(ascending=False) #Adds ranking

In [87]:
def parse(news):
    words = [nltk.word_tokenize(news)]
    words = [item for sublist in words for item in sublist] 
    words = [word.lower() for word in words if (word.isalpha() and len(word)) > 2]
    return words
    

def buidIndex(txt):
    I = {}
    n = 0
    for news in txt:
        T = parse(news)
        for token in set(T):
            if(token not in I.keys()):
                I[token] = [(n, T.count(token))]
            else:
                I[token].append((n, T.count(token)))
        n += 1
    return I

invIndex = buidIndex(txt)

## QUESTÃO 1

Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que? (20 pts.)

In [88]:
def mim(na, nb, nab):
    return nab/na*nb
def emim(na, nb, nab, N):
    if(nab == 0):
        return 0
    return nab*(math.log(N*(nab/na*nb)))
def chi(na, nb, nab, N):
    if(na == 0 or nb ==0):
        return 0
    return (nab-1/N*na*nb)**2/na*nb
def dice(na, nb, nab):
    return nab/na+nb

In [89]:
def printTopMetrics(a):
    best_mim = []
    best_emim = []
    best_chi = []
    best_dice = []
    N = len(txt)
    for b in no_stops:
        if(a == b):
            continue
        na = len(invIndex[a])
        nb = len(invIndex[b])
        nab = len(list(set(invIndex[a]).intersection(invIndex[b])))
        best_mim.append((b, mim(na, nb, nab)))
        best_emim.append((b, emim(na, nb, nab, N)))
        best_chi.append((b, chi(na, nb, nab, N)))
        best_dice.append((b, dice(na, nb, nab)))
    best_mim.sort(key = operator.itemgetter(1))
    best_emim.sort(key = operator.itemgetter(1))
    best_chi.sort(key = operator.itemgetter(1))
    best_dice.sort(key = operator.itemgetter(1))
    print("\nMIM", a, ":")
    [print(f) for f,s in best_mim[-10:]]
    print("\nEMIM", a, ":")
    [print(f) for f,s in best_emim[-10:]]
    print("\nCHI", a, ":")
    [print(f) for f,s in best_chi[-10:]]
    print("\nDICE", a, ":")
    [print(f) for f,s in best_dice[-10:]]
    return [f for f,s in best_chi[-10:]]

In [90]:
stop = set(stopwords.words('portuguese'))
no_stops = [t for t in set(words) if t not in stop]

list_words = ["lista", "com", "varias", "gato", "mano"]
for a in list_words:
    printTopMetrics(a)



MIM lista :
parte
anos
segundo
contra
pode
menos
desde
ter
dois
ainda

EMIM lista :
menos
desde
dois
muitos
quase
passado
relação
disso
bem
mesma

CHI lista :
porque
primeiro
outros
dia
ainda
durante
país
anos
sobre
ser

DICE lista :
todos
parte
segundo
dois
país
ter
ainda
sobre
anos
ser

MIM com :
disse
três
segundo
dois
país
nesta
sobre
anos
ser
ainda

EMIM com :
país
antes
tempo
três
disse
sobre
nesta
anos
ser
ainda

CHI com :
todos
parte
segundo
dois
país
ter
ainda
sobre
anos
ser

DICE com :
todos
parte
segundo
dois
país
ter
ainda
sobre
anos
ser

MIM varias :
diz
primeiro
antes
além
tempo
durante
desde
país
sobre
anos

EMIM varias :
diz
primeiro
antes
além
tempo
durante
desde
país
sobre
anos

CHI varias :
após
então
duas
sempre
cada
passado
dois
ter
ainda
ser

DICE varias :
todos
parte
segundo
dois
país
ter
ainda
sobre
anos
ser

MIM gato :
britânico
euros
vídeo
então
caso
mundo
grande
vez
segundo
ter

EMIM gato :
britânico
euros
vídeo
então
caso
mundo
grande
vez
segundo
ter

CHI g

In [91]:
list_words = ["bolsonaro", "estado", "inglaterra"]
for a in list_words:
    printTopMetrics(a)


MIM bolsonaro :
segundo
primeiro
pode
dois
menos
sobre
presidente
ser
país
ainda

EMIM bolsonaro :
pessoas
além
agora
pode
menos
país
ainda
primeiro
presidente
alguns

CHI bolsonaro :
contra
segundo
desde
dois
país
ter
ainda
sobre
ser
anos

DICE bolsonaro :
parte
todos
segundo
dois
país
ter
ainda
sobre
anos
ser

MIM estado :
outros
anos
todos
ter
sobre
dois
ser
ainda
parte
desde

EMIM estado :
menos
agora
durante
ter
ainda
todos
outros
dois
parte
desde

CHI estado :
porque
presidente
governo
ter
segundo
ainda
país
sobre
anos
ser

DICE estado :
todos
parte
segundo
dois
país
ter
ainda
sobre
anos
ser

MIM inglaterra :
assim
sido
fazer
ser
desde
todos
pouco
dois
outros
ainda

EMIM inglaterra :
fazer
desde
todos
dois
ainda
iniciativa
direitos
sido
pouco
outros

CHI inglaterra :
parte
ano
governo
tempo
três
presidente
ser
país
sobre
anos

DICE inglaterra :
parte
todos
segundo
dois
país
ter
ainda
sobre
anos
ser


* Fiz novas consultas com os termos "bolsonaro", "estado" e "inglaterra" paramelhor avaliar as métricas.
* Para mim a métrica CHI SQUARE é a melhor, por ter retornado palavras mais semelhantes às pesquisadas. Por exemplo: 
    * (lista, primeiro)
    * (bolsonaro, país)
    * (estado, presidente)
    * (inglaterra, governo) 

## QUESTÃO 2

De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez) expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a qualidade dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta. (25 pts.)

In [92]:
def termAtTime(Q, Inv, k):
    L = { key: Inv[key] for key in Q if key in Inv.keys() } #Filtra entrys de interesse no IndexInvertido
    R = {}
    for term in L.keys(): # Para cada termo
            for doc in L[term]: #  Itera pelos docs que tem o termo
                # Acumula o score
                R[doc[0]] = doc[1]
                
                
    #Return K
    if(k > len(R.keys())):
        k = len(R.keys())
    return (sorted(R, key=lambda k: R[k], reverse=True))[:k]

termAtTime("lista".split(), invIndex, 3)

[129, 194, 36]

In [93]:
def returnChi(a, k):
    best_chi = []
    N = len(txt)
    for b in no_stops:
        if(a == b):
            continue
        na = len(invIndex[a])
        nb = len(invIndex[b])
        nab = len(list(set(invIndex[a]).intersection(invIndex[b])))
        best_chi.append((b, emim(na, nb, nab, N)))
    best_chi.sort(key = operator.itemgetter(1))
    return [f for f,s in best_chi[-k:]]

In [94]:
list_words = ["lista", "com", "varias", "gato", "mano"]
for a in list_words:
    for k in [3, 5, 10]:
        query = [a]
        print(query)
        print(termAtTime(query, invIndex, 3))
        query = query + returnChi(a, k)
        print(query)
        print(termAtTime(query, invIndex, 3))


['lista']
[129, 194, 36]
['lista', 'disso', 'bem', 'mesma']
[57, 197, 62]
['lista']
[129, 194, 36]
['lista', 'passado', 'relação', 'disso', 'bem', 'mesma']
[57, 165, 197]
['lista']
[129, 194, 36]
['lista', 'menos', 'desde', 'dois', 'muitos', 'quase', 'passado', 'relação', 'disso', 'bem', 'mesma']
[57, 197, 62]
['com']
[248, 62, 68]
['com', 'anos', 'ser', 'ainda']
[150, 165, 244]
['com']
[248, 62, 68]
['com', 'sobre', 'nesta', 'anos', 'ser', 'ainda']
[210, 248, 234]
['com']
[248, 62, 68]
['com', 'país', 'antes', 'tempo', 'três', 'disse', 'sobre', 'nesta', 'anos', 'ser', 'ainda']
[248, 5, 194]
['varias']
[247]
['varias', 'país', 'sobre', 'anos']
[248, 5, 194]
['varias']
[247]
['varias', 'durante', 'desde', 'país', 'sobre', 'anos']
[248, 5, 194]
['varias']
[247]
['varias', 'diz', 'primeiro', 'antes', 'além', 'tempo', 'durante', 'desde', 'país', 'sobre', 'anos']
[2, 161, 222]
['gato']
[196]
['gato', 'vez', 'segundo', 'ter']
[108, 157, 171]
['gato']
[196]
['gato', 'mundo', 'grande', 'vez', 

In [95]:
list_words = ["bolsonaro", "juíza"]
for a in list_words:
    for k in [3]:
        query = [a]
        print(query)
        print(termAtTime(query, invIndex, 3))
        query = query + returnChi(a, k)
        print(query)
        print(termAtTime(query, invIndex, 3))

['bolsonaro']
[150, 206, 165]
['bolsonaro', 'primeiro', 'presidente', 'alguns']
[18, 215, 224]
['juíza']
[0, 1]
['juíza', 'defesa', 'militar', 'estado']
[114, 6, 248]


In [96]:
print("MELHOR RANK PARA QUERY SOLITARIA BOLSONARO")
print(txt[150][:550])
print("--MELHOR RANK PARA QUERY EXPANDIDA BOLSONARO")
print(txt[18][:550])

print()

print("MELHOR RANK PARA QUERY SOLITARIA JUÍZA")
print(txt[0][:550])
print("--MELHOR RANK PARA QUERY EXPANDIDA JUÍZA")
print(txt[114][:550])

MELHOR RANK PARA QUERY SOLITARIA BOLSONARO
Quando soube que  havia sido assassinada  eu tinha acabado de chegar de Anapu  a cidade que recebeu o sangue de . Quatro tiros tinham arrebentado a cabeça bonita de Marielle e também aquele sorriso que fazia com que mesmo eu  que nunca a conheci  tivesse vontade de rir com ela. Ainda hoje tenho quando vejo a sua fotografia. E rio com Marielle. E então lembro o horror da destruição literal do seu sorriso. E então eu não choro. Eu escrevo.Quando a notícia chegou eu ainda estava na Amazônia  mas me preparava para pegar um avião para . Eu carregava
--MELHOR RANK PARA QUERY EXPANDIDA BOLSONARO
Em apenas dois meses de Governo  o Brasil se tornou o laboratório do novo autoritarismo.  mostrou que pretende governar não por planejamento nem por projetos  não por estudos e cálculos bem fundamentados nem por amplos debates com a sociedade  mas sim pelos urros de quem pode urrar nas . O presidente já fritou pelo menos um ministro e tomou decisões a partir 

Os documentos retornados mudam bastante com a expansão de query, no entanto não consegui ver grande melhora ou piora na qualidade dos resultados apenas lendo as notícias melhor rankeadas. 

No caso da query Juíza a expansão de query piorou a qualidade do resultado. Na query solitária o documento melhor rankeado fala sobre uma decisão de uma juíza enquanto que na query expandida a notícia melhor rankeada fala sobre o ministro do esporte.